<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#导入相关的包" data-toc-modified-id="导入相关的包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关的包</a></span></li><li><span><a href="#读取给定的训练集文件的实体" data-toc-modified-id="读取给定的训练集文件的实体-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>读取给定的训练集文件的实体</a></span></li><li><span><a href="#预测测试集文件中实体的属性" data-toc-modified-id="预测测试集文件中实体的属性-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>预测测试集文件中实体的属性</a></span><ul class="toc-item"><li><span><a href="#获取测试集文件中的实体部分" data-toc-modified-id="获取测试集文件中的实体部分-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>获取测试集文件中的实体部分</a></span></li><li><span><a href="#对分完词的摘要进行情感定位" data-toc-modified-id="对分完词的摘要进行情感定位-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>对分完词的摘要进行情感定位</a></span></li><li><span><a href="#情感分析" data-toc-modified-id="情感分析-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>情感分析</a></span></li><li><span><a href="#获得预测出来的实体的情感类别" data-toc-modified-id="获得预测出来的实体的情感类别-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>获得预测出来的实体的情感类别</a></span></li><li><span><a href="#形成最终需要提交的文件" data-toc-modified-id="形成最终需要提交的文件-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>形成最终需要提交的文件</a></span></li></ul></li></ul></div>

# 导入相关的包 

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import jieba
import time
import re
import math
import json
import xgboost as xbg
from tqdm import tqdm
warnings.filterwarnings("ignore")
os.chdir("E:\\ccf\\Entity_Extraction")

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 读取给定的训练集文件的实体

In [7]:
data_list = []
with open("final\\DATA\\DATA_T_RESULT.txt",encoding="utf-8",mode="r") as rf:
    lines = rf.readlines()
    for line in lines:
        data_list.append(eval(line))
data_pd = pd.DataFrame()
for dl in data_list:
    pd_dl = pd.DataFrame(dl.get("entities"))
    data_pd = pd.concat([data_pd,pd_dl])
print(data_pd.shape)
data_pd.head(5)

(14141, 4)


,digest,eventLevel,keywords,name
0,上海市食品药品监督管理局抽检8大类食品，抽检样品共计1107批次，其中合格1103批次、不合...,中性,,上海市食品药品监督管理局
0,中国质量新闻网讯 据成都市工商行政管理局官网发布的2017年二季度学生用品质量抽检结果，“金...,负向,,汕头市潮南区峡山万年星文具厂
1,中国质量新闻网讯 据成都市工商行政管理局官网发布的2017年二季度学生用品质量抽检结果，“金...,负向,,浙江金丝猴文具有限公司
0,中国网财经8月2日讯 唐山市食品药品监督管理局官网近日发布关于河北省食品安全监督抽检(唐...,负向,,唐山乡伊香生态农业开发有限公司
1,中国网财经8月2日讯 唐山市食品药品监督管理局官网近日发布关于河北省食品安全监督抽检信息的公...,中性,,唐山市食品药品监督管理局


In [10]:
data1_list = []
with open("raw_data\\DATA_1018\\DATA_1018\\DATA_T_RESULT.txt",encoding="utf-8",mode="r") as rf:
    lines = rf.readlines()
    for line in lines:
        data1_list.append(eval(line))
data1_pd = pd.DataFrame()
for dl in data1_list:
    pd_dl = pd.DataFrame(dl.get("entities"))
    data1_pd = pd.concat([data1_pd,pd_dl])
print(data1_pd.shape)
data1_pd.head(5)

(14170, 4)


,digest,eventLevel,keywords,name
0,2017年第二季度，全省共完成并公布 监督 抽检 食品样品3061批次（其中 食用农产品...,中性,,山西省食品药品监督管理局
0,7月14日和17日，青岛市食品药品监督管理局官网公示2017年流通环节第三期食品安全监督...,负向,,青岛崂山大润发商业有限公司
1,7月14日和17日，青岛市食品药品监督管理局官网公示2017年流通环节第三期食品安全监督抽检...,中性,,青岛市食品药品监督管理局
2,7月14日和17日，青岛市食品药品监督管理局官网公示2017年流通环节第三期食品安全监督抽检...,负向,,青岛金鼎广场有限公司
0,施乐辉医用产品国际贸易有限公司召回髋关节置换系统、捷迈医疗国际贸易有限公司召回金属髓内钉手术...,负向,,强生医疗器械有限公司


In [ ]:
enties = list(set(list(data1_pd.name.values) + list(data_pd.name.values)))
with open("feature_data\\enties.txt",mode="w",encoding="utf-8") as wf:
    for e in enties:
        wf.write(e + "\n")


14

11

15

11

13

...

14

13

23

13

10

In [8]:
data_pd[data_pd.name.apply(lambda x: x[-1] == "局")].eventLevel.values

array(['中性', '中性', '中性', ..., '中性', '中性', '中性'], dtype=object)

* 获取原始文件所出现的实体和它的属性，之后形成一个大的字典；

# 预测测试集文件中实体的属性

## 获取测试集文件中的实体部分

In [ ]:
pred_data_list = []
with open("final\\result_with_people_12_09.txt",encoding="utf-8",mode="r") as rf:
    lines = rf.readlines()
    for line in lines:
       pred_data_list.append(eval(line))
pre_data_pd = pd.DataFrame()
for dl in pred_data_list:
    pd_dl = pd.DataFrame(dl.get("entities"))
    pre_data_pd = pd.concat([pre_data_pd,pd_dl])
pre_data_pd.shape
pre_data_pd.head(5)
"宁波优贝车业" in pre_data_pd.name.values

In [11]:
pre_data_pd.isnull().sum()

digest    0
name      0
dtype: int64

获取测试集文件的实体，以及摘要，利用字典匹配的方法计算该实体的情感度

In [12]:
def init_quzao(line):
    return re.sub(r'[,。、]+','###',line) 
def strQ2B(ustring):
    rstring = ""
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:  # 全角空格直接转换
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
            inside_code -= 65248
        rstring += chr(inside_code)
    return rstring

In [13]:
%%time
pre_digest = pre_data_pd.digest.values
digests = []
for digest in pre_digest:
    digest_handle = strQ2B(digest)
    digest_list = init_quzao(digest_handle).split("###")
#     载入用户自定义字典
    jieba.load_userdict("feature_data\\user_dict.txt")
#     对每个句子进行分词
    content = []
    for dl in digest_list:
        seg_word_list = jieba.lcut(dl)
    # 只取分词过后的中文的词，改进的地方是加入标点，因为标点也有情感的意思
        seg_list_without_biaodian = []
        # 匹配分完词之后的中文部分
        for fl in seg_word_list:
            pattern = re.compile(u"[\u4e00-\u9fa5]+")
            result = re.findall(pattern,fl)
            if result != "":
                seg_list_without_biaodian.extend(result)
        content.extend(seg_list_without_biaodian)
    digests.append(content)
digests[0:5]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\adc\AppData\Local\Temp\jieba.cache
Loading model cost 2.821 seconds.
Prefix dict has been built succesfully.


Wall time: 3min 52s


In [14]:
digests[0]
len(digests)

['山东省', '桓台县', '食', '药监局', '抽检', '批次', '食品', '批次', '不合格']

10745

## 对分完词的摘要进行情感定位
对所有的摘要进行分完词之后形成了digests形式，接下来对digests里面每个元素进行情感词定位，并且计算出情感词的情感。接下来首先要获取情感字典

In [15]:
%%time
sentiments_words = pd.read_excel("raw_data\\DATA_1018\\DATA_1018\\NEGATIVE_WORDS.xls")
sentiments_words["正向负向面"] = sentiments_words["正向负向面"].apply(lambda x: 1 if x == "正向" else (-1 if x == "负向" else 0))
words = sentiments_words.关键字.values
sentiment = sentiments_words.正向负向面.values
sent_words_dict = {}
for w_index in range(len(words)):
    if words[w_index] not in sent_words_dict:
        sent_words_dict[words[w_index]] = sentiment[w_index]
    else:
        continue
negative_word_list = []
with open("feature_data\\negative.txt",encoding="utf-8") as rf:
    lines = rf.readlines()
    for line in lines:
        negative_word_list.append(line.strip())

Wall time: 407 ms


In [16]:
"字典长度,", len(sent_words_dict)
sentiments_words.head(5)
negative_word_list

('字典长度,', 338)

,序号,关键字,正向负向面
0,1,经济审判,-1
1,2,民主,1
2,3,合同纠纷,-1
3,4,撤回起诉,-1
4,5,被投诉,-1


['非', '无']

## 情感分析
获取了情感字典，之后就可以开始情感定位了

In [17]:
%%time 
digests_emotion = []
count = 0
for di in tqdm(digests):
    emotion_level = []  # 情感level的list 记录该句子中所有情感词的权重
    pre_word_index = 0
    for word_index in range(len(di)):
        # 如果情感词在字典中，检查和前面情感词之间的否定词的个数，获取该情感词的权重 
        if di[word_index] in sent_words_dict.keys(): 
            neg_count = 0 
            for neg_index in range(pre_word_index,word_index):
                if di[neg_index] in negative_word_list:
                    neg_count += 1
            emotions = sent_words_dict.get(di[word_index]) * math.pow((-1),neg_count)
            emotion_level.append(emotions)
            pre_word_index = word_index
        else:
            continue
    if len(emotion_level) == 0: 
        digests_emotion.append(0)
    else:
        e_l = 0
        for i in emotion_level:
            e_l += i
        digests_emotion.append(e_l) 
digests_emotion[0:10]
len(digests_emotion)
digests_emotion

100%|█████████████████████████████████████████████████████████████████████████| 10745/10745 [00:00<00:00, 19167.42it/s]


Wall time: 639 ms


预测出来的实体的情感类别

In [18]:
digests_emotion[0:10]

[-1.0, 4.0, 0, 0.0, -1.0, 1.0, 1.0, 2.0, 2.0, 1.0]

## 获得预测出来的实体的情感类别

In [19]:
pre_data_pd["pred"] = digests_emotion
pre_data_pd["nature"] = pre_data_pd["pred"].apply(lambda x: "正向" if x > 0 else ("负向" if x < 0 else "中性"))
name_list = pre_data_pd.name.values
pre_data_pd.head()
pre_data_pd.shape
"宁波优贝车业" in name_list

,digest,name,pred,nature
0,山东省桓台县食药监局抽检187批次食品 3批次不合格。,山东省桓台县食药监局,-1.0,负向
1,中国质量新闻网讯 根据《中华人民共和国食品安全法》及其实施条例等有关规定和桓台县食品安全监督...,食品药品监督管理局,4.0,正向
2,检验机构为青岛市华测检测技术有限公司。,青岛市华测检测技术有限公司,0.0,中性
3,（二）桓台喜乐佳商贸有限公司销售的标称青岛永成食品有限公司生产的、生产日期/批号为2017...,青岛永成食品有限公司,0.0,中性
4,根据省食药监局《食品安全抽检监测核查处置工作规范》，对上述抽检中发现的不合格产品，执法人员...,省食药监局,-1.0,负向


(10745, 4)

False

In [20]:
# 原始的训练集中实体
data_pd.head()
data_pd.shape

,digest,eventLevel,keywords,name
0,上海市食品药品监督管理局抽检8大类食品，抽检样品共计1107批次，其中合格1103批次、不合...,中性,,上海市食品药品监督管理局
0,中国质量新闻网讯 据成都市工商行政管理局官网发布的2017年二季度学生用品质量抽检结果，“金...,负向,,汕头市潮南区峡山万年星文具厂
1,中国质量新闻网讯 据成都市工商行政管理局官网发布的2017年二季度学生用品质量抽检结果，“金...,负向,,浙江金丝猴文具有限公司
0,中国网财经8月2日讯 唐山市食品药品监督管理局官网近日发布关于河北省食品安全监督抽检(唐...,负向,,唐山乡伊香生态农业开发有限公司
1,中国网财经8月2日讯 唐山市食品药品监督管理局官网近日发布关于河北省食品安全监督抽检信息的公...,中性,,唐山市食品药品监督管理局


(14141, 4)

In [21]:
name_sor_list = list(data_pd.name.values)
pred_sor_name_list = list(pre_data_pd.name.values)
len(name_sor_list)
len(pred_sor_name_list)
# name_sor_list
# pred_sor_name_list
"宁波优贝车业" in pred_sor_name_list
all_list = pred_sor_name_list + name_sor_list
print(len(set(all_list)))

14141

10745

False

11135


In [22]:
%%time
source_dict = {}
name_list = data_pd.name.values
eventLevel_list = data_pd.eventLevel.values
for index in range(len(name_list)):
    eventLevel_list_dict = {}
    eventLevel_list_dict[eventLevel_list[index]] = 1
    if name_list[index] not in source_dict:
        source_dict[name_list[index]] = eventLevel_list_dict
    else:
        eventLevel_value_dict = source_dict.get(name_list[index])
        if eventLevel_list[index] in eventLevel_value_dict.keys():
            eventLevel_value_dict[eventLevel_list[index]] += 1
        else:
            eventLevel_value_dict[eventLevel_list[index]] = 1
        source_dict[name_list[index]] = eventLevel_value_dict

pred_name_list = pre_data_pd.name.values
pred_eventLevel_list = pre_data_pd.nature.values
for index in range(len(pred_name_list)):
    eventLevel_list_dict = {}
    eventLevel_list_dict[pred_eventLevel_list[index]] = 1
    if pred_name_list[index] not in source_dict:
        source_dict[pred_name_list[index]] = eventLevel_list_dict
    else:
        eventLevel_value_dict = source_dict.get(pred_name_list[index])
        if pred_eventLevel_list[index] in eventLevel_value_dict.keys():
            eventLevel_value_dict[pred_eventLevel_list[index]] += 1
        else:
            eventLevel_value_dict[pred_eventLevel_list[index]] = 1
        source_dict[pred_name_list[index]] = eventLevel_value_dict
source_dict

Wall time: 64.5 ms


从原始的情感字典以及预测出来实体的情感字典中合并，找出出现次数最多的，则就是这个字典的情感分类别

In [23]:
# source_dict.get('山西省食品药品监督管理局')
# source_dict.get('青岛崂山大润发商业有限公司')
# source_dict.get('青岛市食品药品监督管理局')
# source_dict.get('青岛金鼎广场有限公司')
# source_dict.get('强生医疗器械有限公司')
# source_dict.get('捷迈医疗国际贸易有限公司')
# source_dict.get('施乐辉医用产品国际贸易有限公司')
# source_dict.get('碧迪医疗器械有限公司')
# 宁波优贝车业
print(source_dict.get("宁波优贝车业"))
len(source_dict)

None


11135

In [24]:
%%time
result_dict = {}
for k in source_dict.keys():
    if len(source_dict.get(k)) == 1:
        result_dict[k] = list(source_dict.get(k).keys())[0]
    else:
        event_dict = source_dict.get(k)

        sorted_event_dict = sorted(event_dict,key= lambda x: x[1],reverse=True)

        result_dict[k] = sorted_event_dict[0]
all_result_dict = {}
for k,v in result_dict.items():
    if "局" in k:
        all_result_dict[k] = "中性"
    else:
        all_result_dict[k] = v
all_result_dict

Wall time: 32.5 ms


In [25]:
all_result_dict
len(all_result_dict)

{'上海市食品药品监督管理局': '中性',
 '汕头市潮南区峡山万年星文具厂': '负向',
 '浙江金丝猴文具有限公司': '负向',
 '唐山乡伊香生态农业开发有限公司': '负向',
 '唐山市食品药品监督管理局': '中性',
 '上海市工商局': '中性',
 '武汉市家家捞食品有限公司': '负向',
 '一度有限公司': '负向',
 '一度（中国）有限公司': '负向',
 '东莞市袋鼠皮具有限公司': '负向',
 '平湖市嘉信箱包有限公司': '负向',
 '广州市工商局': '中性',
 '乌兰察布市食品药品监督管理局': '中性',
 '内蒙古自治区食品药品监督管理局': '中性',
 '呼和浩特市海莺食品有限公司': '中性',
 '呼和浩特市食品药品监督管理局': '中性',
 '吉林省食药监局': '中性',
 '亳州市豫皖酒业有限公司': '中性',
 '安徽永辉超市有限公司淮北国购广场分公司': '负向',
 '安徽省检验检疫科学技术研究院': '中性',
 '安徽省雅奇药业有限公司': '中性',
 '安徽省食品药品检验研究院': '中性',
 '安徽省食品药品监督管理局': '中性',
 '阜阳市产品质量监督检验所': '中性',
 '黄山市味特佳食品厂': '负向',
 '上海食药监局': '中性',
 '湖北金贵中药饮片有限公司': '负向',
 '安徽拓维检测服务有限公司': '中性',
 '湘潭豪井园食品有限公司': '负向',
 '北京方兴食品有限公司': '负向',
 '武汉万达东湖置业有限公司万达嘉华酒店': '负向',
 '湖北古福泉酒业股份有限公司': '中性',
 '湖北洪森实业（集团）有限公司': '中性',
 '湖北省食品药品监督检验研究院': '中性',
 '湖北省食品质量安全监督检验研究院': '中性',
 '湖北省食药监局': '中性',
 '朔州市公共资源交易中心': '中性',
 '北京市食品药品监督管理局': '中性',
 '国家食品药品监督管理总局': '中性',
 '福州市市场监督管理局': '中性',
 '河市场水产二厅92号': '负向',
 '唐山市食药监局': '中性',
 '市南区万惠家超市': '负向',
 '市南区源香斋食品店': '负向',
 '建希龙摊位': '负向',

11135

In [26]:
all_result_dict.get('山西省食品药品监督管理局')
all_result_dict.get('青岛崂山大润发商业有限公司')
all_result_dict.get('青岛市食品药品监督管理局')
all_result_dict.get('青岛金鼎广场有限公司')
all_result_dict.get('强生医疗器械有限公司')
all_result_dict.get('捷迈医疗国际贸易有限公司')
all_result_dict.get('施乐辉医用产品国际贸易有限公司')
all_result_dict.get('碧迪医疗器械有限公司')
# 高新区食药监局
all_result_dict.get("高新区食药监局")


'中性'

'负向'

'中性'

'负向'

'负向'

'负向'

'负向'

'负向'

## 形成最终需要提交的文件

In [27]:
%%time
result_data = []
result_file = "final\\result_with_people_gf.txt"
with open(result_file,encoding="utf-8") as rf:
    lines = rf.readlines()
    for line in lines:
        result_data.append(eval(line))
for rd in result_data:
    entities = rd.get("entities")
    for e in entities:
        name = e.get("name")
        digist = e.get("digest")
        e["eventLevel"] = all_result_dict.get(e.get("name"))
        e["keywords"] = ""
        del e["name"]
        del e["digest"]
        e["name"] = name
        e["digest"] = digist
    rd["entities"] = entities 
date = time.strftime("%m_%d",time.localtime(time.time()))
file_name = 'submission\\'+ "final_baseline_" + date +".txt" 
# file_name = 'submission\\final_baseline_12_6.txt'  # 通过扩展名指定文件存储的数据为json格式
with open(file_name,'w', encoding="utf-8") as file_object:
    for result in result_data:
        json.dump(result,file_object,ensure_ascii=False)
        file_object.write("\n")

Wall time: 819 ms
